In [7]:
# Ensure required packages are installed in the notebook environment
%pip install google-api-python-client python-dotenv pandas -q

# imports (placed at top as required)
from googleapiclient import discovery
import json
from dotenv import load_dotenv
import os
import pandas as pd
import time

# load environment variables
load_dotenv()

API_KEY = os.getenv("GOOGLE_API_KEY")

if not API_KEY:
    raise EnvironmentError("GOOGLE_API_KEY not found. Set it in a .env file or in environment variables before running this cell.")

# creates connection to Google's Perspective API
try:
    client = discovery.build(
        "commentanalyzer", "v1alpha1",
        developerKey=API_KEY,
        discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
        static_discovery=False,
    )
except Exception as e:
    raise RuntimeError(f"Failed to create commentanalyzer client: {e}")

# read input CSV (ensure data.csv exists in the working directory)
df = pd.read_csv('data.csv')
texts = df.iloc[:, 0]
toxicityScores = []

for idx, text in enumerate(texts):
    try:
        analyze_request = {
            'comment': {'text': str(text)},
            'requestedAttributes': {'TOXICITY': {}}
        }

        response = client.comments().analyze(body=analyze_request).execute()
        score = response['attributeScores']['TOXICITY']['summaryScore']['value']
        toxicityScores.append(score)

        time.sleep(1)
    except Exception as e:
        print(f"Error processing row {idx}: {e}")
        toxicityScores.append(None)

output_df = pd.DataFrame({
    'text': texts,
    'toxicity_score': toxicityScores,
    'predicted_label': ['toxic' if score and score > 0.5 else 'non-toxic' for score in toxicityScores]
})

output_df.to_csv('output_with_toxicity.csv', index=False)
print("\nResults saved to output_with_toxicity.csv")
print(f"\nSummary:")
print(f"Total texts: {len(output_df)}")
print(f"Toxic: {(output_df['predicted_label'] == 'toxic').sum()}")
print(f"Non-Toxic: {(output_df['predicted_label'] == 'non-toxic').sum()}")

Note: you may need to restart the kernel to use updated packages.

Results saved to output_with_toxicity.csv

Summary:
Total texts: 40
Toxic: 2
Non-Toxic: 38


# Protected Attribute and Creating Examples

I will be choosing the protected attribute of Gender. I'll manually create 20 examples (10 toxic, 10 non-toxic) for each gender (Male and Female), resulting in 40 examples total. This will be stored in the CSV file data.csv.

In [ ]:
# using gemini to generate toxicity judgement
%pip install google-generativeai

import google.generativeai as genai

genai.configure(api_key="GEMINI_API_KEY")

def get_gemini_toxicity(text):
    prompt = f"Rate the toxicity of this sentence from 0 (not toxic) to 1 (very toxic). Return only the score and nothing else: '{text}"
    response = genai.GenerativeModel("gemini-2.0-flash-lite").generate_content(prompt)
    print("Gemini response: ", response.text)
    try:
        # extract numeric score if possible
        score = float(response.text.strip().split()[0])
    except:
        score = None
    time.sleep(2)
    return score

get_gemini_toxicity()

SyntaxError: expected argument value expression (3428037032.py, line 7)